In [ ]:
from klrw.framed_dynkin import FramedDynkinDiagram_with_dimensions
from klrw.klrw_algebra import KLRWAlgebra
from klrw.perfect_complex import (
    KLRWPerfectComplex,
    KLRWProjectiveModule,
    KLRWHomOfPerfectComplexes,
    SummandType,
)
from klrw.stable_envelopes import stable_envelope

DD = FramedDynkinDiagram_with_dimensions(CartanType(["A", 1]))
DD.inject_nodes(scope=globals())

In [ ]:
#check construct and check stable envelope in the setting with only two puctures.
sequence = (V1,)
left_framing = W1
right_framing = W1


In [ ]:
DD[left_framing] += 1
DD[right_framing] += 1
for node in sequence:
    DD[node] += 1

print("Framed quiver:")
print(DD)

KLRW = KLRWAlgebra(
    ZZ,
    DD,
    edge_scaling=True,
    vertex_scaling=True,
    warnings=True,
)

In [ ]:
stab = [stable_envelope(KLRW, left_framing, right_framing, sequence, k) for k in range(len(sequence)+1)]

In [ ]:
R = QQ
KLRW_one_grading = KLRWAlgebra(
    R,
    DD,
    default_vertex_parameter=R.one(),
    default_edge_parameter=R.one(),
    warnings=True,
)

In [ ]:
stab = [st.base_change(KLRW_one_grading) for st in stab]

In [ ]:
d = KLRW_one_grading.KLRWmonomial(state=stab[0].projectives[0][0].state, word = (1,)).degree()
print(d)
rhom = KLRWHomOfPerfectComplexes(stab[1],stab[0][0,d])
ext = rhom.ext(0)
assert len(ext.basis()) == 1
v = ext.basis()[0]
e_brane = v.cone()
print(e_brane)


In [ ]:
from itertools import product

for state, i, shift in product(KLRW_one_grading.state_set(), range(-5, 5), range(-5, 5)):
    projective = KLRWProjectiveModule(state, 0)

    one_term = KLRWPerfectComplex(KLRW_one_grading, differentials={}, projectives={0:[projective]}, check=True)
    rhom = KLRWHomOfPerfectComplexes(one_term,e_brane[shift,i])
    ext = rhom.ext(0)
    for v in ext.basis():
        print(state, shift, i)
        print("------")
        print(v)
        print("------")

In [ ]:
new_sequence = sequence

for node in sequence:
    DD[node] += 1
print(DD)

KRLW_new = KLRWAlgebra(
    R,
    DD,
    warnings=True,
    default_vertex_parameter=R.one(),
    default_edge_parameter=R.one(),
)


In [ ]:
projectives_left = {
    i: [
        KLRWProjectiveModule(
            KRLW_new.state(new_sequence + pr.state.as_tuple()), pr.equivariant_degree
        )
        for pr in projs
    ]
    for i, projs in e_brane.projectives.items()
}

differentials_left = {}
for ind, diff in e_brane.differentials.items():
    differentials_left[ind] = matrix(
        KRLW_new,
        diff.nrows(),
        diff.ncols(),
        sparse=True,
    )
    for (i, j), entry in diff.dict(copy=False).items():
        for braid, coeff in entry:
            differentials_left[ind][i, j] += coeff * KRLW_new.KLRWmonomial(
                state=KRLW_new.state(new_sequence + braid.right_state().as_tuple()),
                word=tuple(x + len(new_sequence) for x in braid.word()),
            )



left = KLRWPerfectComplex(
    KRLW_new, differentials=differentials_left, projectives=projectives_left, check=True
)

left

In [ ]:
projectives_right = {
    i: [
        KLRWProjectiveModule(
            KRLW_new.state(pr.state.as_tuple() + new_sequence), pr.equivariant_degree
        )
        for pr in projs
    ]
    for i, projs in e_brane.projectives.items()
}

differentials_right = {}
for ind, diff in e_brane.differentials.items():
    differentials_right[ind] = matrix(
        KRLW_new,
        diff.nrows(),
        diff.ncols(),
        sparse=True,
    )
    for (i, j), entry in diff.dict(copy=False).items():
        for braid, coeff in entry:
            differentials_right[ind][i, j] += coeff * KRLW_new.KLRWmonomial(
                state=KRLW_new.state(braid.right_state().as_tuple() + new_sequence),
                word=braid.word(),
            )



right = KLRWPerfectComplex(
    KRLW_new, differentials=differentials_right, projectives=projectives_right, check=True
)

right

In [ ]:
rhom = KLRWHomOfPerfectComplexes(left, right)
ext = rhom.ext(0)
assert len(ext.basis()) == 1
v = ext.basis()[0]

v

In [ ]:
rhom = KLRWHomOfPerfectComplexes(right,left)
ext = rhom.ext(0)
assert len(ext.basis()) == 1
u = ext.basis()[0]

u

In [ ]:
rhom_left = KLRWHomOfPerfectComplexes(left, left)
ext_left = rhom_left.ext(0)
hom_left = ext_left.hom_of_graded()

uv_dict = {}
for type_u, entry_u in u:
    for type_v, entry_v in v:
        # shift in general?
        if type_u.domain_grading == type_v.domain_grading + v.parent().shift:
            if type_u.domain_index == type_v.codomain_index:
                product_type = SummandType(
                    domain_grading=type_v.domain_grading,
                    domain_index=type_v.domain_index,
                    codomain_index=type_u.codomain_index,
                )
                if product_type in uv_dict:
                    uv_dict[product_type] += entry_v*entry_u
                else:
                    uv_dict[product_type] = entry_v*entry_u

uv = hom_left._from_dict(uv_dict)
print(uv)

ext_left.if_homotopic_to_scalar(uv)

In [ ]:
rhom_right = KLRWHomOfPerfectComplexes(right, right)
ext_right = rhom_right.ext(0)
hom_right = ext_right.hom_of_graded()

vu_dict = {}
for type_u, entry_u in u:
    for type_v, entry_v in v:
        # shift in general?
        if type_v.domain_grading == type_u.domain_grading + u.parent().shift:
            if type_v.domain_index == type_u.codomain_index:
                product_type = SummandType(
                    domain_grading=type_u.domain_grading,
                    domain_index=type_u.domain_index,
                    codomain_index=type_v.codomain_index,
                )
                if product_type in vu_dict:
                    vu_dict[product_type] += coeff*entry_u*entry_v
                else:
                    vu_dict[product_type] = coeff*entry_u*entry_v

vu = hom_right._from_dict(vu_dict)
print(vu)

ext_right.if_homotopic_to_scalar(vu)